# User
## Import Library and set intial value

In [3]:
import numpy as np
import sys
from Adafruit_IO import MQTTClient
import random
import time
import serial.tools.list_ports

In [4]:
size_of_msg = 5

## Prerpare the random bases
#### Because this is just a demonstration, therefore, we  will use this file to send both data from user A and user B at the same time. This will not affect the fact that user A and user B are using different computer and do not know about each other bases yet.

In [5]:
def Sender_random_bases(size,seed):
    np.random.seed(seed)
    random_bases = np.random.randint(2, size=size) # Here we let bases = 0,1 corresponding to bases X,Z
    return random_bases

In [6]:
random_bases_A = Sender_random_bases(size_of_msg,0)

In [7]:
random_bases_B = Sender_random_bases(size_of_msg,1)

## Protocol for the user to send and receive from intermediate interface
We will use MQTT protocol to illustrate the idea of QKD and we will use MQTT as it can easily be use by multiple users. In this code, we use Adafruit server with 3 feed:
* intermediate-interface
* user-a
* user-b

In the code below, we will illustrate the function of user A although we will send 2 data (another one on behalf of user B for demonstrating purpose). User A will publish to the feed intermediate-interface (same as sending data to intermediate interface) and User A will subcribe to feed user-a in order to get to measured result.

In [8]:
AIO_FEED_ID_1 = "intermediate-interface"
AIO_FEED_ID_A = "user-a"
AIO_FEED_ID_B = "user-b"
AIO_USERNAME  = "bbace"
AIO_KEY       = "aio_bPYP32NOKkrTiko7nYSwDr9YlTfz"

In [9]:
def processData(data):
    global A_measured_result
    global B_bases_buffer
    global size_of_msg
    splitData = data.split(":")
    if(splitData[0] == 'UserA_result'):
        size_of_msg = int(splitData[1])
        c = 1
        for i in range(size_of_msg):
            A_measured_result.append(int(splitData[2][i+c]))
            c += 1
    elif(splitData[0] == 'UserB_bases'):
        size_of_msg = int(splitData[1])
        c = 1
        for i in range(size_of_msg):
            B_bases_buffer.append(int(splitData[2][i+c]))
            c += 1

In [10]:
def connected(client):
    print ("Successfully connected ...")
    client.subscribe (AIO_FEED_ID_A)
def subscribe(client , userdata , mid , granted_qos ):
    print(" Subcribe sucessfully...")

def disconnected( client ):
    print(" Disconnected...")
    sys.exit (1)

#After getting the result, A will annouce to B her measure bases
def message(client , feed_id , payload):
    print("Received data: "+ feed_id +" "+payload )
    processData(payload)
    client.publish(AIO_FEED_ID_B, "UserA_bases:"+str(size_of_msg)+':'+str(random_bases_A))

In [11]:
A_measured_result = []
B_bases_buffer = []

In [12]:
client = MQTTClient (AIO_USERNAME , AIO_KEY)
client.on_connect = connected
client.on_disconnect = disconnected
client.on_message = message
client.on_subscribe = subscribe
client.connect()
client.loop_background()

In [13]:
#Implementing error control
client.publish(AIO_FEED_ID_1, "UserA:"+str(size_of_msg)+':'+ str(random_bases_A))
client.publish(AIO_FEED_ID_1, "UserB:"+str(size_of_msg)+':'+ str(random_bases_B))

Connected to Adafruit IO!
Successfully connected ...
 Subcribe sucessfully...


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\DELL\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\DELL\anaconda3\lib\site-packages\paho\mqtt\client.py", line 3452, in _thread_main
    self.loop_forever(retry_first_connection=True)
  File "C:\Users\DELL\anaconda3\lib\site-packages\paho\mqtt\client.py", line 1779, in loop_forever
    rc = self.loop(timeout, max_packets)
  File "C:\Users\DELL\anaconda3\lib\site-packages\paho\mqtt\client.py", line 1181, in loop
    rc = self.loop_read(max_packets)
  File "C:\Users\DELL\anaconda3\lib\site-packages\paho\mqtt\client.py", line 1572, in loop_read
    rc = self._packet_read()
  File "C:\Users\DELL\anaconda3\lib\site-packages\paho\mqtt\client.py", line 2310, in _packet_read
    rc = self._packet_handle()
  File "C:\Users\DELL\anaconda3\lib\site-packages\pa

Received data: user-a UserA_result:5:['h', '1', 'h', 'h', 'h']


## Creating the key
### After getting the result from the interface, A will announce his/her bases to B and will receive it back from B. After that all we need is to compare and create the key

In [1]:
def create_key(A_bases, B_bases, A_result):
    key = []
    for i in range(len(A_bases)):
        if(A_bases[i] == B_bases[i]):
            if(A_result[i] != 'h'):
                key.append(A_result[i])
    return key